In [1049]:
import pandas as pd
import json
import numpy as np
import datetime

In [130]:
extends_files = []
model_elements = {}

In [699]:
buildings_library = ['Air', 'Airflow', 'Applications', 'BoundaryConditions', 'Controls', 'Electrical', 'Fluid', 'HeatTransfer', 'Media', 'Occupants', 'ThermalZones', 'Utilities', 'Types']

In [1028]:
def get_model_elements(json_op, filename, main_folder, model_elements={}, model_equations={}, extends_files=[]):
    for op in json_op:
        class_definition_list = op.get('class_definition')
        if not class_definition_list is None:
            for class_definition_obj in class_definition_list:
                class_specifier_obj = class_definition_obj.get('class_specifier')
                if not class_specifier_obj is None:
                    long_class_specifier_obj = class_specifier_obj.get('long_class_specifier')
                    if not long_class_specifier_obj is None:
                        composition_obj = long_class_specifier_obj.get('composition')
                        if not composition_obj is None:
                            element_list_obj = composition_obj.get('element_list')
                            equation_section_obj_list = composition_obj.get('equation_section')
                            prefixed_element_obj_list = composition_obj.get('prefixed_element')
                            
                            if not element_list_obj is None:
                                element_obj_list = element_list_obj.get('element')
                                for element_obj in element_obj_list:
                                    if 'extends_clause' in element_obj:
                                        extends_file_name = element_obj.get('extends_clause', {}).get('name')
                                        if not extends_file_name in extends_files:
                                            extends_files.append(extends_file_name)

                                    if 'component_clause' in element_obj:
                                        component_clause_obj = element_obj.get('component_clause')
                                        if not component_clause_obj is None:
                                            type_specifier_obj = component_clause_obj.get('type_specifier')
                                            
                                            if not type_specifier_obj is None:
                                                name = component_clause_obj.get('component_list', {}).get('component_declaration', {})[0].get('declaration', {}).get('name')
                                                model_elements[filename+'.'+name] = {'type':type_specifier_obj, 'obj': element_obj}
                                                
                                                if type_specifier_obj.startswith(main_folder):
                                                    if not type_specifier_obj in extends_files:
                                                        extends_files.append(type_specifier_obj)
                            
                            if not equation_section_obj_list is None:
                                for equation_section_obj in equation_section_obj_list:
                                    equation_obj_list = equation_section_obj.get('equation')
                                    if not equation_obj_list is None:
                                        for equation_obj in equation_obj_list:
                                            connect_clause_obj = equation_obj.get('connect_clause')
                                            if not connect_clause_obj is None:
                                                comp1 = connect_clause_obj.get('component1')
                                                comp2 = connect_clause_obj.get('component2')
                                                comp1_element = comp1[0]
                                                comp2_element = comp2[0]
                                                equation_dict1 = {'connected_to': comp2_element, 'connect_clause_obj': connect_clause_obj}
                                                equation_dict2 = {'connected_to': comp1_element, 'connect_clause_obj': connect_clause_obj}
                                                
                                                if len(comp1) == 2:
                                                    comp1_port = comp1[1]
                                                    equation_dict1.update({'comp1_interface': comp1_port})
                                                    equation_dict2.update({'comp2_interface': comp1_port})
                                                if len(comp2) == 2:
                                                    comp2_port = comp2[1]
                                                    equation_dict1.update({'comp2_interface': comp2_port})
                                                    equation_dict2.update({'comp1_interface': comp2_port})
                                                    
                                                if not comp1_element in model_equations:
                                                    model_equations[comp1_element] = [equation_dict1]
                                                else:
                                                    if not equation_dict1 in model_equations[comp1_element]:
                                                        model_equations[comp1_element].append(equation_dict1)
                                                
                                                if not comp2_element in model_equations:
                                                    model_equations[comp2_element] = [equation_dict2]
                                                else:
                                                    if not equation_dict2 in model_equations[comp2_element]:
                                                        model_equations[comp2_element].append(equation_dict2)
                                
                                if not prefixed_element_obj_list is None:
                                    for prefixed_element_obj in prefixed_element_obj_list:
                                        element_obj_list2 = prefixed_element_obj.get('element')
                                        for element_obj2 in element_obj_list2:
                                            class_definition_obj2 = element_obj2.get('class_definition')
                                            component_clause_obj4 = element_obj2.get('component_clause')
                                            
                                            if not class_definition_obj2 is None:
                                                class_specifier_obj2 = class_definition_obj2.get('class_specifier')
                                                if not class_specifier_obj2 is None:
                                                    long_class_specifier_obj2 = class_specifier_obj2.get('long_class_specifier')
                                                    composition_obj2 = long_class_specifier_obj2.get('composition')
                                                    if not composition_obj2 is None:
                                                        element_list_obj3 = composition_obj2.get('element_list')
                                                        if not element_list_obj3 is None:
                                                            element_obj_list3 = element_list_obj3.get('element')
                                                            for element_obj3 in element_obj_list3:
                                                                component_clause_obj3 = element_obj3.get('component_clause')
                                                                if not component_clause_obj3 is None:
                                                                    type_specifier_obj3 = component_clause_obj3.get('type_specifier')
                                            
                                                                    if not type_specifier_obj3 is None:
                                                                        name = component_clause_obj3.get('component_list', {}).get('component_declaration', {})[0].get('declaration', {}).get('name')
                                                                        model_elements[filename+'.'+name] = {'type':type_specifier_obj3, 'obj': element_obj3}
                                                            
                                            if not component_clause_obj4 is None:
                                                type_specifier_obj4 = component_clause_obj4.get('type_specifier')
                                            
                                                if not type_specifier_obj4 is None:
                                                    name = component_clause_obj4.get('component_list', {}).get('component_declaration', {})[0].get('declaration', {}).get('name')
                                                    model_elements[filename+'.'+name] = {'type':type_specifier_obj4, 'obj': element_obj2}
                                                            
                                                            
    return model_elements, model_equations, extends_files

In [1029]:
filename = "Buildings.Examples.VAVReheat.Guideline36"
main_folder = 'Buildings.Examples.VAVReheat'
files_to_parse = [filename]
done_files = []
extends_files = []
model_elements = {}
model_equations = {}

while len(files_to_parse) != 0:
    files_list = files_to_parse
    for filename in files_list:
        with open('raw-json/'+filename+'.json') as fp:
            json_op = json.load(fp)
        model_elements, model_equations, extends_files = get_model_elements(json_op, filename, main_folder, model_elements, model_equations, extends_files)
        files_to_parse.remove(filename)
        if filename in extends_files:
            extends_files.remove(filename)
        done_files.append(filename)
        for file in extends_files:
            if file not in done_files:
                if file.startswith("Buildings") and not file in files_to_parse:
                    files_to_parse.append(file)

In [1030]:
modelica_brick_sensor_type_map = {
    'TemperatureTwoPort': ':Temperature_Sensor',
    'Temperature': ':Temperature_Sensor',
    'RelativeTemperature': ':Temperature_Sensor',
    'TemperatureWetBulbTwoPort': ':Temperature_Sensor',
    'VolumeFlowRate': ':Flow_Sensor',
    'RelativeHumidity': ':Humditiy_Sensor',
    'RelativeHumidityTwoPort': ':Humditiy_Sensor',
    'Pressure': ':Pressure_Sensor',
    'RelativePressure': ':Pressure_Sensor'
}

modelica_brick_heat_exchanger_type_map = {
    'DryCoilCounterFlow': ':Heating_Coil',
    'DryCoilDiscretized': ':Heating_Coil',
    'DryCoilEffectivenessNTU': ':Heating_Coil',
    'WetCoilCounterFlow': ':Cooling_Coil',
    'WetCoilDiscretized': ':Cooling_Coil',
    'EvaporatorCondenser': ':Heat_Exchanger',
    'Heater_T': ':Space_Heater',
}


modelica_brick_actuator_type_map = {
    'Dampers.Exponential': ':Damper', 
    'Dampers.MixingBox': ':Damper', 
    'Dampers.MixingBoxMinimumFlow': ':Damper',
    'Dampers.PressureIndependent': ':Damper',
    'Valves.ThreeWayEqualPercentageLinear': ':Valve',
    'Valves.ThreeWayLinear': ':Valve',
    'Valves.ThreeWayTable': ':Valve',
    'Valves.TwoWayEqualPercentageLinear': ':Valve',
    'Valves.TwoWayLinear': ':Valve',
    'Valves.TwoWayPolynomial': ':Valve',
    'Valves.TwoWayPressureIndependent': ':Valve',
    'Valves.TwoWayQuickOpening': ':Valve',
    'Valves.TwoWayTable': ':Valve'
}

modelica_brick_mover_type_map = {
    'FlowControlled_dp': ':Pump',
    'FlowControlled_m_flow': ':Pump',
    'SpeedCotnrolled_Nrpm': ':Fan',
    'SpeedControlled_y': ':Fan'
}

modelica_brick_thermal_zone_type_map = {
    'Detailed.MixedAir': ':HVAC_Zone',
    'ReducedOrder.EquivalentAirTemperature': ':HVAC_Zone',
    'ReducedOrder.RC': ':HVAC_Zone',
    'ReducedOrer.SolarGain': ':HVAC_Zone'
}

modelica_brick_medium_type_map = {
    'Media.Air': ':Air',
    'Media.Water': ':Water'
}

In [1031]:
# group:
# -1: rest
# 0: sensor
# 1: heat_exchanger
# 2: equpiment
# 3: zone
# 4: medium
# 5: user defined class

In [1032]:
elements_df = pd.DataFrame(model_elements).T
elements_df['brick_type'] = np.NaN
elements_df['group'] = -1
elements_df['element_name']=elements_df.index.to_series().str.rsplit('.', expand=True, n=1)[1]

In [1033]:
def assign_brick_type(df, type_list, type_map, group_id):
    elements_df = df.copy(deep=True)
    for modelica_type in type_list:
        subset_df = elements_df.loc[(elements_df.type.str.startswith(modelica_type))]
        if not subset_df.empty:
            elements_df.loc[subset_df.index, 'brick_type'] = subset_df.type.str.split(modelica_type+'.', expand=True)[1].map(type_map)
            elements_df.loc[subset_df.index, 'group'] = group_id
    
    return elements_df

In [1034]:
elements_df = assign_brick_type(df=elements_df, type_list=['Buildings.Fluid.Sensors', 'Fluid.Sensors'], type_map=modelica_brick_sensor_type_map, group_id=0)
elements_df = assign_brick_type(df=elements_df, type_list=['Buildings.Fluid.HeatExchangers', 'Fluid.HeatExchangers'], type_map=modelica_brick_heat_exchanger_type_map, group_id=2)
elements_df = assign_brick_type(df=elements_df, type_list=['Buildings.Fluid.Actuators', 'Fluid.Actuators'], type_map=modelica_brick_actuator_type_map, group_id=2)
elements_df = assign_brick_type(df=elements_df, type_list=['Buildings.Fluid.Movers', 'Fluid.Movers'], type_map=modelica_brick_mover_type_map, group_id=2)
elements_df = assign_brick_type(df=elements_df, type_list=['Buildings.ThermalZones', 'ThermalZones'], type_map=modelica_brick_thermal_zone_type_map, group_id=3)

In [1047]:
model_equations['cor']

[{'comp1_interface': 'yVAV',
  'comp2_interface': 'yDam',
  'connect_clause_obj': {'component1': ['cor', 'yVAV'],
   'component2': ['conVAVCor', 'yDam']},
  'connected_to': 'conVAVCor'},
 {'comp1_interface': 'yVal',
  'comp2_interface': 'yVal',
  'connect_clause_obj': {'component1': ['cor', 'yVal'],
   'component2': ['conVAVCor', 'yVal']},
  'connected_to': 'conVAVCor'},
 {'comp1_interface': 'y_actual',
  'comp2_interface': 'yDam_actual',
  'connect_clause_obj': {'component1': ['cor', 'y_actual'],
   'component2': ['conVAVCor', 'yDam_actual']},
  'connected_to': 'conVAVCor'},
 {'comp1_interface': 'port_a',
  'comp2_interface': 'port_3',
  'connect_clause_obj': {'component1': ['splSupRoo1', 'port_3'],
   'component2': ['cor', 'port_a']},
  'connected_to': 'splSupRoo1'},
 {'comp1_interface': 'port_b',
  'comp2_interface': 'port_a',
  'connect_clause_obj': {'component1': ['cor', 'port_b'],
   'component2': ['TSupCor', 'port_a']},
  'connected_to': 'TSupCor'},
 {'comp1_interface': 'surf_su

In [1036]:
brick_relationships = []
df_list = []
for element, row in elements_df.loc[elements_df.type.str.startswith(main_folder)].iterrows():
    element_type = row['type']
    element_class = element[:element.rindex('.')]
    element_name = row['element_name']
    for part_element, part_element_row in elements_df.loc[elements_df.index.str.startswith(element_type)].iterrows():
        new_element = element+'.'+part_element_row['element_name']
        new_element_obj = part_element_row['obj']
        new_element_type = part_element_row['type']
        new_element_brick_type = part_element_row['brick_type']
        new_element_name = new_element.split(element_class+'.')[1]
        new_elem_df = pd.DataFrame({new_element: {'obj': new_element_obj, 'type': new_element_type, 'brick_type': new_element_brick_type, 'group': part_element_row['group'], 'element_name': new_element_name}}).T
        df_list.append(new_elem_df)
        
        if not part_element_row.isna().any():
            relationship = {'obj1': element_name, 'relationship': ':hasPart', 'obj2': element_name+part_element.split(element_type)[1]}
            if not relationship in brick_relationships:
                brick_relationships.append(relationship)
                elements_df.loc[element, 'group'] = 5

In [1037]:
elements_df = pd.concat([elements_df] + df_list, axis=0, sort=False)
new_elements_df = pd.concat(df_list)
for element_idx, row in new_elements_df.iterrows():
    element_name = row['element_name']
    parent_element = element_name.split('.')[0]
    old_name = element_name.split('.')[1]
    equations = model_equations.get(old_name, [])
    for equation in equations:
        comp2_df = elements_df.loc[elements_df.element_name == parent_element+'.'+equation.get('connected_to').split('[')[0]]
        if comp2_df.empty:
            comp2_df = elements_df.loc[elements_df.element_name == equation.get('connected_to').split('[')[0]]
        new_equation = {'connected_to': comp2_df.element_name.values[0], 'connect_clause_obj': {'component1': ['vav', 'port_b'], 'component2': ['senMasFlo', 'port_a']}, 'comp1_interface': 'port_b', 'comp2_interface': 'port_a'}
        
        if not element_name in model_equations:
            model_equations[element_name] = [new_equation]
        else:
            model_equations[element_name].append(new_equation)
        comp2_type = comp2_df.type.values[0]

In [1038]:
for relationship in elements_df.dropna().apply(lambda x: {'obj1': x.element_name, 'relationship': ':type', 'obj2': x.brick_type}, axis=1).values:
    if not relationship in brick_relationships:
        brick_relationships.append(relationship)

In [1045]:
def search_for_equipment(element, prev_elements, elements_df, model_equations):
    element = element.split('[')[0]
        
    if elements_df.loc[elements_df.element_name == element].empty:
        print("can't find element "+element)
    else:
        element_type = elements_df.loc[elements_df.element_name == element].type.values[0]
    
#     print("searching for equipment for "+element+ " type = "+element_type)
#     print("path traversed = ",prev_elements)
    
    if element not in model_equations:
#         print("element does not have any connections; returning None")
        return None
    
    if element in prev_elements:
#         print("element already traversed; returning None")
        return None
    
    if element_type.startswith('Buildings.Controls'):
#         print("not traversing a controller; returning None")
        return None
    
    if element_type.startswith("Buildings.Fluid.Sources"):
#         print("not handling sources right now, returning None")
        return None
    
    found = False
    equations = model_equations[element]
    
    connected_equipments = []
    for equation in equations:
        connected_to_element = equation['connected_to'].split('[')[0]
        comp1_port = equation.get('comp1_interface')
#         print(equation)
#         print(element, comp1_port)
        
        connected_element_df = elements_df.loc[elements_df.element_name == connected_to_element]
        if connected_element_df.empty:
            print('cant find element '+connected_to_element)
        elif not comp1_port is None and (comp1_port.startswith("port_a") or comp1_port.startswith("port_1")):
#             print("ignored")
            pass
        elif connected_to_element in prev_elements:
            pass
        elif connected_element_df.group.values[0] == 2 or connected_element_df.group.values[0] == 5:
            if connected_to_element not in connected_equipments:
                connected_equipments = connected_equipments + [connected_to_element]
        else:
            if not element in prev_elements:
                prev_list2 = prev_elements+[element]
            closest_equipment = search_for_equipment(connected_to_element, prev_list2, elements_df, model_equations)
            if not closest_equipment is None:
                connected_equipments = list(set(connected_equipments+ closest_equipment))
            prev_elements = prev_list2
        prev_elements.append(connected_to_element)
    return connected_equipments

In [1046]:
equipment_df = elements_df.loc[elements_df.group == 2]
for element_idx, row in equipment_df.iterrows():
    element_name = row['element_name']
    next_equipment_list = search_for_equipment(element_name, [], elements_df, model_equations)
#     print(element_name, next_equipment_list)
    
    for next_equipment in next_equipment_list:
        next_equipment_group = elements_df.loc[elements_df.element_name == next_equipment].group.values[0]
        if next_equipment_group == 2:
            relationship = {'obj1': element_name, 'relationship': ':feeds', 'obj2': next_equipment}
            brick_relationships.append(relationship)

In [1043]:
brick_relationships

[{'obj1': 'cor', 'obj2': 'cor.vav', 'relationship': ':hasPart'},
 {'obj1': 'cor', 'obj2': 'cor.terHea', 'relationship': ':hasPart'},
 {'obj1': 'sou', 'obj2': 'sou.vav', 'relationship': ':hasPart'},
 {'obj1': 'sou', 'obj2': 'sou.terHea', 'relationship': ':hasPart'},
 {'obj1': 'eas', 'obj2': 'eas.vav', 'relationship': ':hasPart'},
 {'obj1': 'eas', 'obj2': 'eas.terHea', 'relationship': ':hasPart'},
 {'obj1': 'nor', 'obj2': 'nor.vav', 'relationship': ':hasPart'},
 {'obj1': 'nor', 'obj2': 'nor.terHea', 'relationship': ':hasPart'},
 {'obj1': 'wes', 'obj2': 'wes.vav', 'relationship': ':hasPart'},
 {'obj1': 'wes', 'obj2': 'wes.terHea', 'relationship': ':hasPart'},
 {'obj1': 'flo', 'obj2': 'flo.sou', 'relationship': ':hasPart'},
 {'obj1': 'flo', 'obj2': 'flo.eas', 'relationship': ':hasPart'},
 {'obj1': 'flo', 'obj2': 'flo.nor', 'relationship': ':hasPart'},
 {'obj1': 'flo', 'obj2': 'flo.wes', 'relationship': ':hasPart'},
 {'obj1': 'flo', 'obj2': 'flo.cor', 'relationship': ':hasPart'},
 {'obj1': 

In [ ]:
                    rec = {
                        'id': ident,
                        'source': type(self).__name__,
                        'record': {
                            'encoding': 'XML',
                            'content': subtree,
                        },
                        'triples': [(NS[ident], RDF.type, rdflib.URIRef(brick_class))],
                        'timestamp': timestamp
                    }

In [1050]:
records = []
for relationship in brick_relationships:
    brick_record = {
        "timestamp": datetime.datetime.utcnow(),
        "triples": [
            relationship['obj1'], relationship['relationship'], relationship['obj2']
        ]
    }
    records.append(brick_record)
records

[{'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433868),
  'triples': ['cor', ':hasPart', 'cor.vav']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433874),
  'triples': ['cor', ':hasPart', 'cor.terHea']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433876),
  'triples': ['sou', ':hasPart', 'sou.vav']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433877),
  'triples': ['sou', ':hasPart', 'sou.terHea']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433878),
  'triples': ['eas', ':hasPart', 'eas.vav']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433879),
  'triples': ['eas', ':hasPart', 'eas.terHea']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433881),
  'triples': ['nor', ':hasPart', 'nor.vav']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433882),
  'triples': ['nor', ':hasPart', 'nor.terHea']},
 {'timestamp': datetime.datetime(2020, 7, 18, 17, 6, 36, 433883),
  'triples': ['wes', ':has

# Not handled yet

#elements_df.loc[((elements_df.type.str.startswith('Buildings')) & (elements_df.brick_type.isna()) & ~(elements_df.type.str.startswith(main_folder)))]